## Import Libraries

In [1]:
from time import sleep

In [2]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [3]:
import matplotlib.pyplot as plt
plt.ion()

In [4]:
from pydgilib_extra import *

Set the path to `dgilib.dll`.

In [5]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

## Data Logging

Create a figure for the plot.

In [6]:
fig = plt.figure(figsize=(10, 6))
fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [7]:
config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "gpio_delay_time" : 0.00075,
    "fig": fig,
    "verbose": 0,
}

Perform the measurement.

In [8]:
data = []
fig.clf()

with DGILibExtra(dgilib_path, **config_dict) as dgilib:
    dgilib.device_reset()
    dgilib.logger_start()
    sleep(1)
    dgilib.update_callback()
    while not all(dgilib.data[INTERFACE_GPIO][1][-1]):
        dgilib.update_callback()
#         sleep(1)
    sleep(0.1)
    dgilib.logger_stop()
    data = dgilib.data

## Analysis

In [9]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, 1)

power_and_time_per_pulse done, charges: 3000, times: 3000


In [10]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3)

In [11]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(nsc_store_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(nsc_store_charge) - 1}")

MAX_NUM_BYTES: 3000


In [12]:
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
#data

{48: [[0.0019254666666666668,
   0.0019254666666666668,
   0.023723333333333336,
   0.023723333333333336,
   0.02381986666666667,
   0.02381986666666667,
   0.04401986666666667,
   0.04401986666666667,
   0.0440764,
   0.0440764,
   0.0642796,
   0.0642796,
   0.06434573333333334,
   0.06434573333333334,
   0.0845484,
   0.0845484,
   0.0846076,
   0.0846076,
   0.1048124,
   0.1048124,
   0.10488066666666666,
   0.10488066666666666,
   0.12508546666666667,
   0.12508546666666667,
   0.12514733333333333,
   0.12514733333333333,
   0.14535320000000002,
   0.14535320000000002,
   0.14542466666666667,
   0.14542466666666667,
   0.16563053333333333,
   0.16563053333333333,
   0.16569506666666667,
   0.16569506666666667,
   0.18591746666666667,
   0.18591746666666667,
   0.18600813333333333,
   0.18600813333333333,
   0.2062188,
   0.2062188,
   0.20629026666666667,
   0.20629026666666667,
   0.22650466666666666,
   0.22650466666666666,
   0.22658040000000002,
   0.22658040000000002,
   0.2

In [13]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [14]:
results = []
for y in [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3000
    # variables        = 2
    chi-square         = 4.3868e-07
    reduced chi-square = 1.4632e-10
    Akaike info crit   = -67933.6087
    Bayesian info crit = -67921.5960
[[Variables]]
    slope:      8.9899e-07 +/- 2.5501e-10 (0.03%) (init = 0)
    intercept:  1.7113e-05 +/- 4.4181e-07 (2.58%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 3000
    # variables        = 2
    chi-square         = 4.1830e-07
    reduced chi-square = 1.3953e-10
    Akaike info crit   = -68076.3295
    Bayesian info crit = -68064.3168
[[Variables]]
    slope:      9.0383e-07 +/- 2.4902e-10 (0.03%) (init = 0)
    intercept:  1.5559e-05 +/- 4.3142e-07 (2.77%) (init = 1)
[[Correlations]] (unrep

In [15]:
fig2 = plt.figure(figsize=(9, 8))

In [16]:
charge_color = 'r'
time_color = 'b'

In [17]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [mC]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [18]:
lines = []
lines += ax1.plot(num_bytes, nsc_store_charge, charge_color+'-', label='TrustZone Store Charge')
lines += ax1.plot(num_bytes, nsc_load_charge, charge_color+'--', label='TrustZone Load Charge')
lines += ax2.plot(num_bytes, nsc_store_time, time_color+'-', label='TrustZone Store Time')
lines += ax2.plot(num_bytes, nsc_load_time, time_color+'--', label='TrustZone Load Time')
ax1.legend(handles=lines)
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
ax1.set_title(f"TrustZone Store Charge: Base {results[0].params['intercept'].value:.03} mC plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Load Charge: Base {results[1].params['intercept'].value:.03} mC plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Store Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"TrustZone Load Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [19]:
# data